In [1]:
import pandas as pd
import numpy as np
from collections import Counter

# Random data generation

We define parameters for the data; these parameters must be changed in relation to the number of workshops and the number of students per room.

In [2]:
MAX_AAL_B = 4*70
MAX_AAL_M = 6*70
MAX_ESB_B = 2*50
MAX_ESB_M = 2*50
MAX_CPH_B = 2*50
MAX_CPH_M = 2*50
MAX_ESB_TOTAL = 2*50

NUM_STUDENTS = MAX_AAL_B+MAX_AAL_M+MAX_CPH_B+MAX_CPH_M+MAX_ESB_TOTAL 

In [3]:
print(NUM_STUDENTS)

1000


In [4]:
df = pd.read_csv("fixed_data.csv", sep=",", encoding="latin1")
df = df[df["ID"] > 60] # this is because the first rows were incorrect

We set the seed, but it can be removed to generate different samples and not always the same one.

In [5]:
np.random.seed(639)

Now, we set the number of data points we want to create

In [6]:
n_sint = NUM_STUDENTS

We create a dataframe where we will have the new data; these data will be generated with different combinations of "categories"

In [7]:
df_sint = pd.DataFrame()

# 2.1. New ID (that does not match the original)
df_sint["ID"] = range(1, n_sint + 1)

In [8]:
print(df["registration_started"].head(20))

11    2/5/25 10:15:03
12    2/5/25 10:21:33
13    2/5/25 10:30:21
14    2/5/25 10:47:07
15    2/5/25 10:52:21
16    2/5/25 10:54:59
17    2/5/25 11:14:30
18    2/5/25 11:12:36
19    2/5/25 11:46:00
20    2/5/25 12:16:57
21    2/5/25 12:24:04
22    2/5/25 12:30:44
23    2/5/25 10:42:14
24    2/5/25 12:05:59
25    2/5/25 12:44:22
26    2/5/25 12:59:49
27    2/5/25 13:06:37
28    2/5/25 13:57:41
29    2/5/25 13:56:38
30    2/5/25 13:54:15
Name: registration_started, dtype: object


To generate more realistic data, the times will also be generated at random

In [9]:
df.columns = df.columns.str.strip()

# Convert time columns to datetime
for col in ["registration_started", "registration_finished"]:
    # Depends on the Excel format, this changes
    # df[col] = pd.to_datetime(df[col], format="%d-%m-%Y %H:%M", errors="coerce")
    df[col] = pd.to_datetime(df[col], format="%m/%d/%y %H:%M:%S", errors="coerce")

# Actual duration in seconds
real_duration = (
    df["registration_finished"] - df["registration_started"]
).dt.total_seconds().dropna()

# ----------------- SYNTHETIC TIMES -----------------
start_base = df["registration_started"].dropna().sample(
    n_sint, replace=True
).reset_index(drop=True)

dur_base = real_duration.sample(n_sint, replace=True).reset_index(drop=True)

jitter_start = np.random.normal(loc=0, scale=600, size=n_sint)  # ±10 min
jitter_dur   = np.random.normal(loc=0, scale=30,  size=n_sint)  # ±30 s

dur_min = real_duration.min()
dur_max = real_duration.max()

dur_sint = dur_base + jitter_dur
dur_sint = np.clip(
    dur_sint,
    a_min=max(1, dur_min * 0.5),
    a_max=dur_max * 1.5
)

start_sint  = start_base + pd.to_timedelta(jitter_start, unit="s")
finish_sint = start_sint + pd.to_timedelta(dur_sint, unit="s")

df_sint["registration_started"]  = start_sint
df_sint["registration_finished"] = finish_sint

To create the data, note that bachelor students have two columns filled in that master’s students do not.

In [10]:
# We remove columns already processed
df_sint["SUMA"] = 6
cols_exclude = ["ID", "registration_started", "registration_finished", "SUMA"]
without_cols_exclude = [c for c in df.columns if c not in cols_exclude]
select_cols = [col for col in without_cols_exclude if "Select" in col]
num1_cols = [col for col in without_cols_exclude if "Select" not in col and "_1" in col]
others_cols = [col for col in without_cols_exclude if "Select" not in col and "_1" not in col]

In [11]:
df_bachelor = df[df["program"] == "bachelor"]
df_master = df[df["program"] == "master"]
print("---------------Last 2 columns------------\n")
print(f"There are {len(df_bachelor)} bachelor students and {len(df_master)} master students\n")
print("In the second-to-last column there are blank spaces:\n")
print(f"Bachelor: {df_bachelor[select_cols[-2]].isna().sum()}, Master: {df_master[select_cols[-2]].isna().sum()}")
print("\nIn the last column there are blank spaces:\n")
print(f"Bachelor: {df_bachelor[select_cols[-1]].isna().sum()}, Master: {df_master[select_cols[-1]].isna().sum()}")

---------------Last 2 columns------------

There are 306 bachelor students and 468 master students

In the second-to-last column there are blank spaces:

Bachelor: 0, Master: 468

In the last column there are blank spaces:

Bachelor: 0, Master: 468


Therefore, whether they are bachelor or master students matters when filling in the columns.

In [12]:
j = 0
for i in range(len(without_cols_exclude)):
    col = without_cols_exclude[i]
    col_data = df[col].dropna()
    col_data_master = df_master[col].dropna()
    col_data_bachelor = df_bachelor[col].dropna()

    if col_data.empty:
        # if the column is empty, we fill it with NaN
        df_sint[col] = np.nan
        continue
    
    # To choose campus or program, everyone must have it (i=0 and i=1)
    if i == 0:
        maximum = {
            ("AAL", "bachelor"): MAX_AAL_B,
            ("AAL", "master"): MAX_AAL_M,
            ("CPH", "bachelor"): MAX_CPH_B,
            ("CPH", "master"): MAX_CPH_M,
            ("ESB", "bachelor"): MAX_ESB_B,
            ("ESB", "master"): MAX_ESB_M
        }
        
        counters = Counter({k: 0 for k in maximum})
        results = []
        for _ in range(n_sint):
            candidates = []
            for (campus, program), max_use in maximum.items():
                if counters[(campus, program)] >= max_use:
                    # If it exceeds the maximum, it is not a valid candidate
                    continue
                if campus == "ESB":
                    total_esb = (
                        counters[("ESB", "bachelor")] +
                        counters[("ESB", "master")]
                    )
                    if total_esb >= MAX_ESB_TOTAL:
                        continue

                # If it passes all filters, it is a valid candidate
                candidates.append((campus, program))
            if not candidates:
                raise ValueError("No categories remain that meet the constraints to assign more rows.")

            # We choose a random category among the valid ones
            idx = np.random.randint(len(candidates))
            campus_sel, program_sel = candidates[idx]

            results.append((campus_sel, program_sel))
            counters[(campus_sel, program_sel)] += 1

        # Convertimos a DataFrame con el mismo índice que df_sint
        pairs = np.array(results, dtype=object)
        df_pairs = pd.DataFrame(pairs, columns=["campus", "program"], index=df_sint.index)

        df_sint[["campus", "program"]] = df_pairs

    elif i == 1:
        pass
        
        # For the following columns, everyone must have up to 8 different workshop choices
    elif col in select_cols:
        if j == 0:
            df_sint.loc[df_sint["program"]=="bachelor", col] = np.random.choice(
                df_bachelor[col].dropna().unique(),
                size= (df_sint["program"] == "bachelor").sum(),
                replace=True
            )
            df_sint.loc[df_sint["program"]=="master", col] = np.random.choice(
                df_master[col].dropna().unique(),
                size= (df_sint["program"] == "master").sum(),
                replace=True
            )
        elif j < len(select_cols) - 2:
            categories_bach = df_bachelor[col].dropna().unique()

            for idx in df_sint.index[df_sint["program"] == "bachelor"]:
                # values already used in this row, in the previous columns
                used = set(
                    v for v in df_sint.loc[idx, select_cols[:j]].tolist()
                    if pd.notna(v)
                )

                # available categories (those that are NOT used)
                available = [c for c in categories_bach if c not in used]

                if available:
                    df_sint.at[idx, col] = np.random.choice(available)
                else:
                    df_sint.at[idx, col] = np.nan

            categories_master = df_master[col].dropna().unique()

            for idx in df_sint.index[df_sint["program"] == "master"]:
                used = set(
                    v for v in df_sint.loc[idx, select_cols[:j]].tolist()
                    if pd.notna(v)
                )

                available = [c for c in categories_master if c not in used]

                if available:
                    df_sint.at[idx, col] = np.random.choice(available)
                else:
                    df_sint.at[idx, col] = np.nan

        # In the last 2 columns, only bachelor has programs
        else:
            df_sint.loc[(df_sint["program"]=="bachelor"), col] = np.random.choice(
                df_bachelor[col].dropna().unique(),
                size= (df_sint["program"] == "bachelor").sum(),
                replace=True
            )
        j += 1
        
    elif col in others_cols:
        for a in range(3):
            mask = (df_sint[select_cols[a]].astype(str).str.contains(col, na=False, regex=False))
            df_sint.loc[mask, col] = str(a + 1)
    else:
        df_sint[col] = np.nan

    

# Save the CSV and make sure the columns match
df_sint = df_sint[df.columns]
df_sint.to_csv("new_data.csv", index=False)
